In [0]:
USE CATALOG `price-prediction-analytics`;

CREATE SCHEMA IF NOT EXISTS silver;
DROP TABLE IF EXISTS silver.daily_pricing;
CREATE TABLE IF NOT EXISTS silver.daily_pricing (
DATE_OF_PRICING DATE,
ROW_ID BIGINT,
STATE_NAME STRING,
MARKET_NAME STRING,
PRODUCTGROUP_NAME STRING,
PRODUCT_NAME STRING,
VARIETY STRING,
ORIGIN STRING,
ARRIVAL_IN_TONNES DECIMAL(18,2),
MINIMUM_PRICE DECIMAL(36,2),
MAXIMUM_PRICE DECIMAL(36,2),
MODAL_PRICE DECIMAL(36,2),
source_file_load_date TIMESTAMP,
lakehouse_inserted_date TIMESTAMP,
lakehouse_updated_data TIMESTAMP
)

In [0]:
USE CATALOG `price-prediction-analytics`;

INSERT INTO silver.daily_pricing
SELECT
  to_date(DATE_OF_PRICING, 'MM/dd/yyyy') AS DATE_OF_PRICING,
  cast(ROW_ID as BIGINT) AS ROW_ID,
  STATE_NAME,
  MARKET_NAME,
  PRODUCTGROUP_NAME,
  PRODUCT_NAME,
  VARIETY,
  ORIGIN,
  cast(ARRIVAL_IN_TONNES as DECIMAL(18,2)) AS ARRIVAL_IN_TONNES,
  cast(MINIMUM_PRICE as decimal(36,2)) AS MINIMUM_PRICE,
  cast(MAXIMUM_PRICE as decimal(36,2)) AS MAXIMUM_PRICE,
  cast(MODAL_PRICE as decimal(36,2)) AS MODAL_PRICE,
  try_cast(source_file_load_date as timestamp) AS source_file_load_date,
  current_timestamp() AS lakehouse_inserted_date,
  current_timestamp() AS lakehouse_updated_data
FROM `price-prediction-analytics`.bronze.daily_pricing
WHERE DATE_OF_PRICING != 'DATE_OF_PRICING'  -- remove repeated headers
  AND DATE_OF_PRICING IS NOT NULL;

In [0]:
select * from `price-prediction-analytics`.silver.daily_pricing order by DATE_OF_PRICING desc

In [0]:
Alter TABLE `price-prediction-analytics`.processrunlogs.deltalakehouse_process_runs
ADD COLUMNS (PROCESSED_TABLE_DATETIME TIMESTAMP)

In [0]:
INSERT INTO `price-prediction-analytics`.processrunlogs.deltalakehouse_process_runs(PROCESS_NAME,PROCESSED_TABLE_DATETIME,PROCESS_STATUS)
SELECT 'daily_pricing', max(source_file_load_date), 'Completed' FROM `price-prediction-analytics`.silver.daily_pricing

In [0]:
select * from `price-prediction-analytics`.processrunlogs.deltalakehouse_process_runs order by PROCESsed_table_datetime desc

In [0]:
USE CATALOG `price-prediction-analytics`;
INSERT INTO  silver.daily_pricing
SELECT
to_date(DATE_OF_PRICING,'dd/MM/yyyy'),
cast(ROW_ID as bigint) ,
STATE_NAME,
MARKET_NAME,
PRODUCTGROUP_NAME,
PRODUCT_NAME,
VARIETY,
ORIGIN,
cast(ARRIVAL_IN_TONNES as decimal(18,2)),
cast(MINIMUM_PRICE as decimal(36,2)),
cast(MAXIMUM_PRICE as decimal(36,2)),
cast(MODAL_PRICE as decimal(36,2)),
try_cast(source_file_load_date as timestamp) ,
current_timestamp(),
current_timestamp()
FROM `price-prediction-analytics`.bronze.daily_pricing
WHERE source_file_load_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'daily_pricing_silver' AND process_status = 'Completed' )


In [0]:
select * from `price-prediction-analytics`.silver.daily_pricing  where DATE_OF_PRICING ='2023-02-18'